In [ ]:
# Informações Gerais sobre o funcionamento do Algoritmo:


# O Parâmetro da Função Principal, 'data_e_hora', deve estar no formato " AAAA-MM-DDTHH:MM:SS", exemplo: "2023-11-18T18:00:00"
# O horário utilizado deve ser o horário UTC ( Tempo Universal Coordenado ), se a observação for feita as 15:00:00, o horário UTC deve ser 18:00:00
# OBS: O horário correto da observação deve ser sempre considerado para o ideal funcionamento do código
# Para esse caso, a extensão dos arquivos de imagem é JPG. Caso outra extensão seja utilizada, deve-se alterar a extensão na função principal
# Após rodar o código, deve-se selecionar a pasta que contém as imagens na janela de diálogo que o códiga abrirá
# A depdender do quantitativo de imagens, o código demora cerca de um minuto para fazer o alinhamento
# Após o alinhamento, a imagem resultante será mostrada ao lado da imagem de referência
# Após a exibição da imagem, clique sobre quantas manchas quiser e precione "esc" duas vezes, e será mostrado as coordenadas de cada mancha solar no notebook

In [ ]:
import numpy as np
import cv2
import hvpy
import matplotlib.pyplot as plt
import os
import sunpy

In [ ]:
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from sunpy.coordinates import frames
from sunpy.map import Map
from PIL import Image, ImageOps
from rembg import remove
from scipy.ndimage import rotate

In [ ]:
import tkinter as tk
from tkinter import filedialog
import random

In [ ]:
%matplotlib qt # Essa função mágica é essencial para o correto funcionamento do código.

In [ ]:
# Função principal:

def main(data_e_hora):
    # Busca o diretório cujas imagens estão armazenadas
    diretorio = buscar_diretorio()
    # Cria uma lista de strings contendo os nomes dos arquivos no diretório que possuem a terminação ".JPG"
    strings = [i for i in os.listdir(diretorio) if i.endswith('.JPG')]
    # Faz o download e abertura da imagem referência
    ref = down_ref(data_e_hora)
    # Cria um loop para processar cada imagem disponível no diretório, armazenando-as em uma lista inicialmente vazia
    imagens = []    
    for nome in strings:
        img = abrir_imagem(f'{diretorio}/{nome}')
        img_ref, img_alin = recorte_e_centralizacao(ref, img)
        imagens.append(img_alin)
    # Cria uma nova imagem apartir do empilhamento das imagens alinhadas
    imagem_empilhada = empilhamento(imagens)
    # Alinha a imagem com a imagem de ref através de rotação
    imagem_final, indice, diff_rel = alinhar_por_rotacao(img_ref, imagem_empilhada)
     
    # Obtem as coordenadas heliográficas das manchas solares
    dados_finais = obter_coordenadas(img_ref, imagem_final, data_e_hora)
    #retorna a imagem final, após o processo de alinhamento bem como os dados heliográficos das manchas solares capturadas
    return imagem_final, dados_finais, indice, diff_rel

In [ ]:
# Busca o diretório da pasta que contém as imagens:

def buscar_diretorio():
    # Cria a janela principal do tkinter e janela de diálogo
    root = tk.Tk()
    root.withdraw()
    # Abre a janela de diálogo e busca a pasta das imagens
    pasta_selecionada = filedialog.askdirectory()
    return pasta_selecionada

In [ ]:
# Faz o download da imagem SDO de referência:

def down_ref(data_e_hora):
    # Faz o download do arquivo JPEG2000 diretamente da base de dados do Helioviewer/NASA
    hmi_file = hvpy.save_file(
        hvpy.getJP2Image(Time(data_e_hora).datetime, hvpy.DataSource.HMI_INT.value), "JP2.JPEG2000",  overwrite=True)
    # abre o arquivo JPEG2000
    img = cv2.imread("JP2.JPEG2000")
    #Remove o fundo escuro da imagem
    img = remove(img)
    #Converte a imagem para tons de cinza
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

In [ ]:
def abrir_imagem(caminho):
    img = cv2.imread(caminho)
    img = remove(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return img

In [ ]:
def recorte_e_centralizacao(ref, img):
    # ------------------------------------------------------------- Recorte --------------------------------------------------
    # Imagem SDO de referência
    # _______________________________________________________________________________________________________________________________#
    
    # Binariza a imagem
    ref_copy = cv2.medianBlur(ref, 5)
    ret, thresh1 = cv2.threshold(ref_copy,0,255,cv2.THRESH_BINARY)
    # Identifica pontos de diferença de tonalizade
    edges1 = cv2.Canny(thresh1, 0, 255)
    # Identifica contornos nos pontos de diferença de tonalizade
    contours1, hierarchy = cv2.findContours(edges1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Identifica o maior contorno presente (Que deve, por lógica, ser o disco solar)
    max_contour_terra = max(contours1, key=cv2.contourArea)
    # Obtém as coordenadas e dimensões do retângulo delimitador que envolve o maior contorno.
    x1, y1, w1, h1 = cv2.boundingRect(max_contour_terra)
    # Define, por recorte, a região de interesse
    ri_ref = ref[y1:y1+h1, x1:x1+w1]
    ri_ref = cv2.resize(ri_ref, (1024, 1024))
    
    # Imagem IFE
    # ______________________________________________________________________________________________________________________________#
    
    # Binariza a imagem
    img_copy = cv2.medianBlur(img, 5)
    ret, thresh2 = cv2.threshold(img_copy,0,255,cv2.THRESH_BINARY)
    # Identifica pontos de diferença de tonalizade
    edges2 = cv2.Canny(thresh2, 0, 255)
    # Identifica contornos nos pontos de diferença de tonalizade
    contours2, hierarchy = cv2.findContours(edges2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Identifica o maior contorno presente (Que deve, por lógica, ser o disco solar)
    max_contour_terra = max(contours2, key=cv2.contourArea)
    # Obtém as coordenadas e dimensões do retângulo delimitador que envolve o maior contorno.
    x2, y2, w2, h2 = cv2.boundingRect(max_contour_terra)
    # Define, por recorte, a região de interesse
    ri_img = img[y2:y2+h2, x2:x2+w2]
    ri_img = cv2.resize(ri_img, (1024, 1024))
    
    
    # ------------------------------------------------------ Centralização ------------------------------------------------------

    # Cria uma borda escura ao redor do disco solar recortado
    img1 = Image.fromarray(ri_ref)
    img2 = Image.fromarray(ri_img)
    border_width = 100
    bimg1 = ImageOps.expand(img1, border=border_width, fill='black')
    bimg2 = ImageOps.expand(img2, border=border_width, fill='black')
    # Redimensiona as imagens para as mesmas dimensões
    img_ref = cv2.resize(np.array(bimg1), (1024, 1024))
    img_alin = cv2.resize(np.array(bimg2), (1024, 1024))

    return img_ref, img_alin

In [ ]:
def empilhamento(lista_de_imagens):
    
    imagens_empilhadas = lista_de_imagens
  
    # Calcular a média das imagens
    imagem_final = np.zeros(imagens_empilhadas[0].shape, dtype=np.float32)
    for img in imagens_empilhadas:
        imagem_final += img / len(imagens_empilhadas)
    
    # Converter a imagem final de volta para o tipo uint8
    imagem_final = np.uint8(imagem_final)

    return imagem_final

In [ ]:
def alinhar_por_rotacao(ref, imagem):
    
    # Cria uma lista com 360 imagens, rotacionadas com passo de 1° entre cada imagem
    
    angulos = np.arange(0, 360, 1)   
    lista_rotacoes = [rotate(imagem, theta, reshape=False) for theta in angulos]
    # Calcula a diferença relativa entre cada rotação tomando com base na imagem de referência
    
    Diff_relativa = []

    for i in lista_rotacoes:
        diff = cv2.absdiff(ref, i)
        diff_percent = (np.sum(diff) / np.sum(i)) * 100
        Diff_relativa.append(diff_percent)
        
    # Determina qual das rotações tem menor diferença relativa
    
    indice_da_minima_dif = Diff_relativa.index(min(Diff_relativa))
    imagem_rotacionada = lista_rotacoes[indice_da_minima_dif]

    # retorna a imagem rotacionada
    return imagem_rotacionada, indice_da_minima_dif, Diff_relativa

In [ ]:
def obter_coordenadas(img_ref, imagem, data_e_hora):
    
    def Pixel_to_Stony(x_pix, y_pix, mapa):
        # Determina a data e hora da observação 
        data = mapa.date.value
        # Converte as coordenadas de pixel para WCS (Segundos de arco)
        ptw = mapa.pixel_to_world(x_pix*u.pix, y_pix*u.pix)
        x_arc, y_arc = ptw.Tx, ptw.Ty
        # Cria o sistema de coordenadas helioprojectivas
        c = SkyCoord(x_arc.value*u.arcsec, y_arc.value*u.arcsec, frame=frames.Helioprojective, obstime=data,
                 observer="earth")
        # Transforma o sistema de coordenadas helioprojectivas em Heliográficas Stonyhurst
        c = c.transform_to(frames.HeliographicStonyhurst)
        # Determina a latitude e a longitude heliográficas
        la = c.lat.value
        lo = c.lon.value
    
        return la, lo

    # inverte a imagem (Pois o mapa sunpy inverte as imagens submetidas a elas, assim a imagem original deve ser invertida antes.)
    img = cv2.flip(imagem, 0)
    img_ref = cv2.flip(img_ref, 0)

    # Cria o sistema de coordenadas helioprojectivas 
    skycoord = SkyCoord(0*u.arcsec, 0*u.arcsec, obstime=data_e_hora,
                    observer='earth', frame=frames.Helioprojective)
    # Cria o cabeçalho fit
    header = sunpy.map.make_fitswcs_header(img, skycoord, scale=[2.3, 2.3]*u.arcsec/u.pixel)

    
    # Cria os mapas para comparação
    mapa1 = sunpy.map.Map(img, header)
    mapa2 = sunpy.map.Map(img_ref, header)
    
    # Plota as imagens referencia e alinhada para verificação do procedimento e captura de coordenadas das manchas solares
    
    fig = plt.figure()
    ax1 = fig.add_subplot(121, projection=mapa1)
    ax2 = fig.add_subplot(122, projection=mapa2)
    mapa1.plot(axes=ax1)
    mapa1.draw_limb(color='blue')
    mapa2.plot(axes=ax2)
    mapa2.draw_limb(color='blue')

    while True:
            pts = []
            pts = np.asarray(plt.ginput(n=-1, timeout=-1))
            if plt.waitforbuttonpress():
                break
    plt.close()

    xm, ym = pts[:,0], pts[:,1]

    # Procedimento para obtenção do erro padrão das latitudes e longitudes
    
    Dados = []

    # Loop para processamento em cada par de coordenadas capturadas.
    
    for i in range(len(xm)):
        
        # Define a região quadrada de lado 5 pixels
        lado = 5
        x_clicado = xm[i]
        y_clicado = ym[i]
        regiao_quadrada = [[x_clicado + j, y_clicado + k] for j in range(lado) for k in range(lado)]
        # Obtem-se de forma aleatória, um conjunto de 10 pixels nessa região
        pixels_aleatorios = random.sample(regiao_quadrada, 10)
        pixels_aleatorios = np.array(pixels_aleatorios)

        CX= []
        CY = []

        # Converte-se esses 10 pixels em coordenadas heliográficas stonyhurst
        for l in range(len(pixels_aleatorios[:,0])):
            cs_x, cs_y = Pixel_to_Stony(pixels_aleatorios[:,0], pixels_aleatorios[:,1], mapa1)
            CX.append(cs_x)
            CY.append(cs_y)
            
        # Define a latitude e longitude através da média das 10 coordenadas
        
        Latitude = np.mean(CX)
        Longitude = np.mean(CY)

        # Calcula o erro padrão
        
        epla = np.std(CX, ddof=1)/np.sqrt(len(CX))
        eplo = np.std(CY, ddof=1)/np.sqrt(len(CY))

        Dados.append([mapa1.date.value, Latitude, epla, Longitude, eplo])

    return Dados
    

In [ ]:
data_e_hora = "2023-12-22T18:17:00" # Deve-se sempre determinar a data e hora da obervação que se deseja alinhar e rotacionar

In [ ]:
imagem_final, dados_finais, indice, diff_rel = main(data_e_hora)

In [ ]:
cv2.imwrite(f'{data_e_hora[0:10]}.jpg', imagem_final)

In [ ]:
print('Dados para essa imagem:')
print("----------------------------------------------------------------------------------------------------")
print(f'A rotação da imagem foi de {indice} graus')
for i in dados_finais:
    print(f'Data e Hora: {i[0]}, Latitude: {i[1]:.3f} com erro de {i[2]:.3f}, Longitude: {i[3]:.3f} com erro de {i[4]:.3f}')